# Import & Path

## Import

In [ ]:
import numpy as np
import cv2
import os
import random

## Path

In [ ]:
basePath = os.getcwd()
trainPath = os.path.join(os.getcwd(), "Images")
testPath = os.path.join(os.getcwd(), "Online-Test-sort")
print(trainPath)
print(testPath)

# Default Configuration

In [ ]:
height = 128
width = 128

random.seed(42)
np.random.seed(42)

# Split & Enhance

## Create Function

In [ ]:
def enhance_data(data_dir, enhance_type = "none", test_data = False, robust = False):
    for category in os.listdir(data_dir):
        if (robust):
            max = len(os.listdir(os.path.join(data_dir, category))) - 1
            max = max + 30 - (max % 30)
            max = max / 30
            unique = random.randint(0, (max-1))
            unique = str(unique)
            unique = unique.zfill(5)

        for file in os.listdir(os.path.join(data_dir, category)):
            img = cv2.imread(os.path.join(data_dir, category, file))

            if img is not None:
                img = cv2.resize(img, (height, width))
                clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
                kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])

                save_path = os.path.join(basePath, (enhance_type + "-train"), category)

                match enhance_type:
                    case "gaussian":
                        img = cv2.GaussianBlur(img, (5, 5), 0)
                    case "median":
                        img = cv2.medianBlur(img, 5)
                    case "bilateral":
                        img = cv2.bilateralFilter(img, 9, 75, 75)
                    case "sharpen":
                        img = cv2.filter2D(img, -1, kernel)
                    case "clahe":
                        # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        # img = clahe.apply(img)
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
                        img[:,:,0] = clahe.apply(img[:,:,0])
                        img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
                    case "combo":
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
                        img[:,:,0] = clahe.apply(img[:,:,0])
                        img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
                        img = cv2.GaussianBlur(img, (5, 5), 0)
                    case "none":
                        pass
                    case _:
                        raise ValueError(f"Unknown enhance type: {enhance_type}")
                    
                if (robust):
                    if (file.startswith(unique)):
                        save_path = os.path.join(basePath, "robust-test", category)
                    else:
                        save_path = os.path.join(basePath, 'robust-train', category)
                    
                if (test_data):
                    save_path = os.path.join(basePath, enhance_type + "-test", category)
                    
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                    
                condition = cv2.imwrite(os.path.join(save_path, file), img)

                if (not condition):
                    print(os.path.join(save_path, file), condition)

## Process Dataset

Add some necessary enhance data for training models.

In the current research we use 4 type of enhancement as mention below:

|No|Type|
|--|----|
|1|Gaussian|
|2|Sharpen|
|3|Clahe|
|4|Clahe & Gaussian|

In order to create a robust model we also create some condition where we split 1 unique train dataset as test data and the rest as train data. All the data use gaussian blur.

### Enhance Dataset

In [ ]:
enhance_data(trainPath, 'gaussian')
enhance_data(trainPath, 'sharpen')
enhance_data(trainPath, 'clahe')
enhance_data(trainPath, 'combo')

### Robust Dataset

Create a robust model with test data taken from train data. With the configuration of each class, 1 set of unique images is taken for test data and the rest are for train data

In [ ]:
enhance_data(trainPath, 'gaussian', robust = True)

### Test Dataset

In [ ]:
enhance_data(testPath, 'gaussian', test_data = True)
enhance_data(testPath, 'sharpen', test_data = True)
enhance_data(testPath, 'clahe', test_data = True)
enhance_data(testPath, 'combo', test_data = True)